## Connect to mongodb - pymongo


1. 
    pip install pymongo[srv]
    or 
    pip install pymongo

2. pip install dnspython

3. Mongodb have several sample collection, in this code we will use sample_airbnb collection.

4. Convert mongodb json document into table format(csv)

6. Cleaning that data 

7. Applying Data visualization concept.


In [ ]:
from pymongo import MongoClient

In [ ]:
from pprint import pprint
# pprint library which is not related to MongoDB 
#but is used here only to make the output structured and visually appealing from a console.

In [ ]:
CONNECTION_STRING = "mongodb+srv://<userName>:<password>@<clusterName>.mongodb.net/sample_airbnb"
client = MongoClient(CONNECTION_STRING)

If getting an error , Restart Kernal! 


------------> RESTART KERNAL <-------------

In [ ]:
db = client['sample_airbnb']

In [ ]:
collections = db.list_collection_names()
print ("collections:", collections, "\n")
# Print all the collection present in sample_airbnb

In [ ]:
collection = db["listingsAndReviews"]

In [ ]:
collection.find_one()

In [ ]:
query=[
    {
        '$project':
        {
            'accommodates':1,
            '_id':0
        }
    }
]
l=collection.aggregate(query)
l2=[]
for i in l:
  #  print(i["accommodates"])
    l2.append(i["accommodates"])

l2

In [ ]:
import seaborn as sns

In [ ]:
sns.countplot(l2)

In [ ]:
print(collection.distinct('address.country'))

What if we want to work with csv rather than json. 
For the purpose of data visalization.

In [ ]:
new_df=[]
for x in collection.find({},{"_id":0,"name":1,"bedrooms":1,"beds":1,"price":1,
                             "review_scores.review_scores_accuracy":1,
                            "review_scores.review_scores_cleanliness":1,
                            "review_scores.review_scores_checkin":1,
                            "review_scores.review_scores_communication":1,
                            "review_scores.review_scores_location":1,
                            "review_scores.review_scores_value":1,
                            "review_scores.review_scores_rating":1}).limit(40):
    new_df.append(x)

In [ ]:
new_df

In [ ]:
from pandas import json_normalize
df = json_normalize(new_df)

In [ ]:
import pandas as pd

Converting onto pandas dataframe so that we can perform it's functions.

In [ ]:
pd.DataFrame(df)

In [ ]:
df.head()

In [ ]:
df=df.rename(columns = {'review_scores.review_scores_accuracy':'accuracy_score', 
                        'review_scores.review_scores_cleanliness':'cleanliness_score',
                        'review_scores.review_scores_checkin':'checkin_score',
                        'review_scores.review_scores_communication':'communication_score',
                        'review_scores.review_scores_location':'location_score',
                        'review_scores.review_scores_value':'value_score',
                        'review_scores.review_scores_rating':'rating_score'})

In [ ]:
df.shape

In [ ]:
df.describe()

While using the describe () we can notice that 'price' column is not present which means it is not saved as numerical value.
We need to convert it into float 

In [ ]:
df.price.dtype

In [ ]:
df["price"] = df.price.astype(str).astype(float)

In [ ]:
df.price.dtype

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

From total 40 rows 15 have nan value

In [ ]:
df.isnull().sum(axis=1)

In [ ]:
df = df.dropna(thresh=7)
# we used thresh = 7 inside the df.dropna() function which means 
#it will drop all those rows where Nan values are 7 or more than 7
    

In [ ]:
# Resetting the indices using df.reset_index()
df = df.reset_index(drop=True)

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
import seaborn as sns

In [ ]:
sns.barplot(x='price',y='accuracy_score', data=df)

In [ ]:
sns.heatmap(df.corr(), center=0, cmap='BrBG', annot=True)